# Task 3: Advanced CNN

We implement and evaluate an additional CNN architecture which has less parameters and further uses dropout- and maxpool-layers.

In [ ]:
from pathlib import Path
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, plot_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD, RMSprop, lr_scheduler
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset

import shap

from data import get_img_dataset
from project3Lib.transforms import EnhanceContrast
import project3Lib.CNN as cnn
from project3Lib.CNN import train_model, test, predict
import project3Lib.utils as utils

from masked_dataset import MaskedDataset

In [ ]:
# load data and transform data if augmentation == yes
augmentation = input("Use augmentation? [yes/no]").lower() == "yes"

unique = input("Use unique images?[yes/no]").lower() == "yes"
input_path = "data/unique_images" if unique else "data/images"

if augmentation:
    transform = [EnhanceContrast(reduce_dim=False), transforms.Grayscale()]
    train_dataset,val_dataset, test_dataset = get_img_dataset(transform, data_path=input_path, use_same_transforms = True)
    transform = [EnhanceContrast(reduce_dim=False), transforms.Grayscale(), transforms.RandomRotation(70), transforms.RandomHorizontalFlip(), transforms.ColorJitter()]
    train_dataset2,val_dataset2, _ = get_img_dataset(transform,data_path=input_path, use_same_transforms = True)
    train_dataset = ConcatDataset([train_dataset,train_dataset2])
    val_dataset = ConcatDataset([val_dataset,val_dataset2,val_dataset2])
    model_file_path = "AdvancedCNN_augmented_unique" if unique else "baselineCNN_augmented"
    
else: 
    train_dataset, val_dataset, test_dataset = get_img_dataset(data_path=input_path)
    
    model_file_path = "AdvancedCNN_unique" if unique else "baselineCNN"
    
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('Device state:', device)
batch_size = 16
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
print(f"Class sizes{np.unique([y for x,y in train_dataset], return_counts = True)}")

dataloaders = {
    'train' : trainloader, 
    'validation': validloader
}

image_datasets = {
    'train': train_dataset,
    'validation': val_dataset
}

In [ ]:
total_trainable_params = sum(
    p.numel() for p in cnn.CNN().parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

# Model Implementation
## Train classifier

In [ ]:
n = 20
accs = []
f1s = []
for i in range(n):
    model = cnn.CNN()
    criterion = CrossEntropyLoss()
    optimizer = RMSprop(model.parameters(), lr=0.0001)
    epochs = 50
    model = train_model(model, criterion, optimizer, dataloaders, image_datasets, 3, num_epochs=epochs)
    model.eval()
    acc, f1 = test(model, test_dataset)
    accs.append(acc)
    f1s.append(f1)

In [ ]:
a, f = test(model, test_dataset)
print(f"Accuracy mean: {np.mean(accs)} std: {np.std(accs)}")
print(f"F1 mean: {np.mean(f1s)} std: {np.std(f1s)}")
print(f"Score of saved model: acc = {a} and f1 = {f}")

```
Accuracy mean: 0.74 std: 0.066332495807108
F1 mean: 0.7591884057971013 std: 0.08237728074604544
```

In [ ]:
torch.save(model.state_dict(), f"{model_file_path}.pt")

## Evaluate Classifier

In [ ]:
model = cnn.CNN()
model.load_state_dict(torch.load(f"{model_file_path}.pt"))
model.eval()

In [ ]:
x_test = [i for i,j in test_dataset]
y_test = [j for i,j in test_dataset]
preds = []
outs = []
for t in x_test:
    pred, out = predict(model, t)
    preds.append(pred)
    
print(f"Accuracy: {accuracy_score(preds,y_test)}")
print(f"F1 score: {f1_score(preds,y_test)}")

cm=confusion_matrix(y_test,preds,normalize="true")
cmd = ConfusionMatrixDisplay(cm)
cmd.plot()

Performance scores

```
Accuracy: 0.85
F1 score: 0.888888888888889
```

![](Plots/CM_CNN.png)

# Interpretability

In [ ]:
test_dataset_nomask = test_dataset
transform = [transforms.Grayscale()]
common_transform = [EnhanceContrast(reduce_dim=False)]
_,_, test_dataset = get_img_dataset(transform = transform, \
                                    use_same_transforms=True, \
                                    common_transforms=common_transform, \
                                    data_path=input_path, \
                                    folder_type = MaskedDataset, \
                                    mask_folder=Path("data/masks"))


## SHAP

In [ ]:
# Deep Explainer
bg = [i for i,j in train_dataset]
bg = torch.stack(bg)
e = shap.DeepExplainer(model, bg)
outs = []
for i in bg:
    pred, out = predict(model,i)
    outs.append((out[0][0].item(), out[0][1].item()))
print(f"Mean values {np.mean([i for i,j in outs])}, {np.mean([j for i,j in outs])}")

In [ ]:
ious = []
for i, (image,mask,target) in enumerate(test_dataset):
    image = image.reshape((1,1,128,128))
    pred, out = predict(model,image)
    
    shap_values = e.shap_values(image)
    shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]
    test_numpy = np.swapaxes(np.swapaxes(image.cpu().numpy(), 1, -1), 1, 2)
    print(f"Image #{i}: True Class {target}, Prediction {pred}, Probabilities {out}")
    shap.image_plot(shap_numpy, test_numpy, labels = ["SHAP for class 0","SHAP for class 1"])
    
    predicted_mask = np.copy(shap_values[1].reshape(128,128))
    mask = mask.numpy().reshape((128,128))
    pixels = int(np.sum(mask.flatten()))
    iou = utils.evaluate_interpretability(predicted_mask, mask,pixels)
    print(iou)
    if target == 1:
        ious.append(iou)

```
The mean iou is 0.3190489056289843
```

# Integrated Gradients with Captum

In [ ]:
ious = []
plots = []
for i, (image,mask,target) in enumerate(test_dataset):
    data = (image,target)
    if target == 1: 
        class_1, class_0 = utils.plot_grads(data,model, plot=False,grad_type= "integ_grads")
    else:
        class_0, class_1 = utils.plot_grads(data,model, plot=False,grad_type= "integ_grads")
    predicted_mask = np.copy(class_1.cpu().detach().numpy().reshape(128,128))
    mask = mask.numpy().reshape((128,128))
    pixels = int(np.sum(mask.flatten()))
    iou = utils.evaluate_interpretability(predicted_mask, mask,pixels)
    if target == 1:
        ious.append(iou)
    if i == 0:
        plots.append(predicted_mask)
    if i == 1:
        plots.append(predicted_mask)
print(f"The mean iou is {np.mean(ious)}")
for i, plot in enumerate(plots): 
    np.save(f"Plots/AdvancedCNN_IntGrad_{i}", plot)

```
The mean iou is 0.3106789354483127
```

In [ ]:
utils.plot_grads_dataloader(test_dataset_nomask, model, grad_type= "integ_grads" ,plot=True, save_name="cnn")

# GradCam

In [ ]:
model.model[4] # last conv layer

In [ ]:
ious = []
plots = []
for i, (image,mask, target) in enumerate(test_dataset):
    data = (image,target)
    
    if target == 1: 
        class_1, class_0 = utils.plot_grads(data,model, layer_idx = 4,plot=False,grad_type= "grad_cam")
    else:
        class_0, class_1 = utils.plot_grads(data,model, layer_idx = 4,plot=False,grad_type= "grad_cam")
    predicted_mask = np.copy(class_1.cpu().detach().numpy().reshape(128,128))
    mask = mask.numpy().reshape((128,128))
    pixels = int(np.sum(mask.flatten()))
    iou = utils.evaluate_interpretability(predicted_mask, mask,pixels)
    if target == 1:
        ious.append(iou)
    if i == 0:
        np.save(f"Plots/AdvancedCNN_GradCam_0", predicted_mask)
    if i == 1:
        np.save(f"Plots/AdvancedCNN_GradCam_1", predicted_mask)
print(f"The mean iou is {np.mean(ious)}")
    

```
The mean iou is 0.2718315831205122
```

In [ ]:
utils.plot_grads_dataloader(test_dataset_og, model, grad_type= "grad_cam" ,plot=True,layer_idx=4, save_name="cnn")

## Appendix: Minimal adjusted CNN

In [ ]:
total_trainable_params = sum(
    p.numel() for p in SmallCNN().parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score


class SmallCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.2),
            nn.Conv2d(32, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.2),
            nn.Flatten(),
            nn.Linear(512, 64),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(64, 2),
            nn.Softmax(dim=-1),
        )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch):
        images, labels = batch
        # Get predictions
        out = self(images)
        # Get loss
        loss = F.cross_entropy(out, labels)
        return loss

    @torch.no_grad()
    def validation_step(self, batch):
        images, labels = batch
        # Get predictions
        out = self(images)
        # Get loss
        loss = F.cross_entropy(out, labels)
        # Get accuracy
        _, preds = torch.max(out, dim=1)
        acc = accuracy_score(labels.cpu(), preds.cpu())
        return {'val_loss': loss, 'val_acc': acc}

In [ ]:
n = 20
accs = []
f1s = []
for i in range(n):
    model = SmallCNN()
    criterion = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.001)
    epochs = 50
    model = train_model(model, criterion, optimizer, dataloaders, image_datasets, 4, num_epochs=epochs)
    acc, f1 = test(model, test_dataset)
    accs.append(acc)
    f1s.append(f1)

In [ ]:
a, f = test(model, test_dataset)
print(f"Accuracy mean: {np.mean(accs)} std: {np.std(accs)}")
print(f"F1 mean: {np.mean(f1s)} std: {np.std(f1s)}")
print(f"Score of saved model: acc = {a} and f1 = {f}")

```
Accuracy mean: 0.7074999999999999 std: 0.057608593109014575
F1 mean: 0.7094406762827815 std: 0.06790884228169164
Score of saved model: acc = 0.7 and f1 = 0.6666666666666666
```

In [ ]:
x_test = [i for i,j in test_dataset]
y_test = [j for i,j in test_dataset]
preds = []
outs = []
for t in x_test:
    pred, out = predict(model, t)
    preds.append(pred)
    
print(f"Accuracy: {accuracy_score(preds,y_test)}")
print(f"F1 score: {f1_score(preds,y_test)}")